In [6]:
import os

In [7]:
!pwd

/c/Users/abhis/OneDrive/Desktop/Projects/end-to-end-cdc/chicken-diseases-classification/research


In [8]:
os.chdir("../")

In [9]:
!pwd

/c/Users/abhis/OneDrive/Desktop/Projects/end-to-end-cdc/chicken-diseases-classification


In [10]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [13]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [15]:
import os
import zipfile
from pathlib import Path
import urllib.request as request
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        # Force redownload by removing the file if it exists
        if os.path.exists(self.config.local_data_file):
            logger.info(f"Removing existing file: {self.config.local_data_file}")
            os.remove(self.config.local_data_file)

        # Download the file
        filename, headers = request.urlretrieve(
            url=self.config.source_URL,
            filename=self.config.local_data_file
        )
        logger.info(f"{filename} downloaded with the following info: \n{headers}")

        # Log file size
        file_size = os.path.getsize(self.config.local_data_file)
        logger.info(f"Downloaded file size: {file_size} bytes")

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        # Check if the file is a valid ZIP file
        if not zipfile.is_zipfile(self.config.local_data_file):
            logger.error(f"The file {self.config.local_data_file} is not a valid ZIP file.")
            raise zipfile.BadZipFile(f"The file {self.config.local_data_file} is not a valid ZIP file.")

        # Extract the ZIP file
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted all files to {unzip_path}")

In [16]:
from cnnClassifier.pipeline.stage_01_data_ingestion import ConfigurationManager
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-08 16:53:05,049: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-08 16:53:05,052: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-08 16:53:05,053: INFO: common: created directory at: artifacts]
[2025-05-08 16:53:05,055: INFO: common: created directory at: artifacts/data_ingestion]
[2025-05-08 16:53:05,057: INFO: 1100761653: Removing existing file: artifacts/data_ingestion/data.zip]
[2025-05-08 16:53:09,416: INFO: 1100761653: artifacts/data_ingestion/data.zip downloaded with the following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B6B5:27E2A0:55B0:9629:681C941C
Accept-Rang